In [62]:
import numpy as np
from sklearn import svm

In [63]:
import numpy as np
from sklearn import svm

In [64]:
training_features = np.loadtxt("features.train")
testing_features = np.loadtxt("features.test")

In [65]:
def build_my_kernel(Q):
    def my_kernel(xn, xm):
        return (1 + np.dot(xn, xm.T)) ** Q
    
    return my_kernel

In [66]:
def generate_one_versus_all_yn(digits, one_choice):
    yn = np.ones(len(digits))
    
    for i, digit in enumerate(digits):
        if digit != one_choice:
            yn[i] = -1
            
    return np.array(yn)

In [67]:
def parse_digits(digits, choice1, choice2):
    result = []
    
    for digit in digits:
        if digit == choice1 or digit == choice2:
            result.append(digit)
            
    return digits

In [68]:
def generate_one_versus_one_yn(digits, choice1, choice2):
    parsed_digits = parse_digits(digits, choice1, choice2)
    yn = np.ones(len(parsed_digits))
    
    for i, digit in enumerate(digits):
        if digit == choice2:
            yn[i] = -1
    
    return yn

In [69]:
def calc_error(yn, y_pred):
    # misclassified points are opposite sign so will be -1 when multiplied
    error = yn * y_pred
    return np.count_nonzero(error == -1) / len(yn)

In [70]:
def split_data(one_versus_all, dataset, choice1, choice2):
    to_return = dataset
    if not one_versus_all:
        to_return = []
        
        for digit, intensity, symmetry in dataset:
            if digit == choice1 or digit == choice2:
                to_return.append([digit, intensity, symmetry])

    to_return = np.array(to_return)
    digits = to_return[:, 0]
    intensity = to_return[:, 1]
    symmetry = to_return[:, 2]
    
    return digits, intensity, symmetry

In [71]:
def my_kernel(xn, xm):
    return np.exp(-1 * np.linalg.norm(xn - xm) ** 2)

In [73]:
def generate_one_versus_all_yn(digits, one_choice):
    yn = np.ones(len(digits))
    
    for i, digit in enumerate(digits):
        if digit != one_choice:
            yn[i] = -1
            
    return np.array(yn)

In [74]:
def parse_digits(digits, choice1, choice2):
    result = []
    
    for digit in digits:
        if digit == choice1 or digit == choice2:
            result.append(digit)
            
    return digits

In [75]:
def generate_one_versus_one_yn(digits, choice1, choice2):
    parsed_digits = parse_digits(digits, choice1, choice2)
    yn = np.ones(len(parsed_digits))
    
    for i, digit in enumerate(digits):
        if digit == choice2:
            yn[i] = -1
    
    return yn

In [76]:
def calc_error(yn, y_pred):
    # misclassified points are opposite sign so will be -1 when multiplied
    error = yn * y_pred
    return np.count_nonzero(error == -1) / len(yn)

In [81]:
def run_svm(one_versus_all, choice, choice1=-1, C=0.01, Q=2):
    digits_train, intensities_train, symmetries_train = split_data(one_versus_all, training_features, choice, choice1)
    digits_test, intensities_test, symmetries_test = split_data(one_versus_all, testing_features, choice, choice1)
    
    if (one_versus_all):
        yn_train = generate_one_versus_all_yn(digits_train, choice)
        yn_test = generate_one_versus_all_yn(digits_test, choice)
    
    else:
        yn_train = generate_one_versus_one_yn(digits_train, choice, choice1)
        yn_test = generate_one_versus_one_yn(digits_test, choice, choice1)

    xn_train = np.column_stack((intensities_train, symmetries_train))
    xn_test= np.column_stack((intensities_test, symmetries_test))
    model = svm.SVC(C=C, kernel='rbf', gamma=1)
    model.fit(xn_train, yn_train)
    
    y_train_pred = model.predict(xn_train)
    y_test_pred = model.predict(xn_test)
    
    Ein = calc_error(yn_train, np.array(y_train_pred))
    Eout = calc_error(yn_test, np.array(y_test_pred))
    
    return Ein, np.sum(model.n_support_), Eout

Q9

In [82]:
C = [0.01, 1, 100, 1e4, 1e6]

for c in C:
    E_in, _, _ = run_svm(False, 1, 5, c)
    print(f"E_in for C={c}: {E_in}")

E_in for C=0.01: 0.003843689942344651
E_in for C=1: 0.004484304932735426
E_in for C=100: 0.0032030749519538757
E_in for C=10000.0: 0.0025624599615631004
E_in for C=1000000.0: 0.0006406149903907751


Q10

In [83]:
for c in C:
    _, _, E_out = run_svm(False, 1, 5, c)
    print(f"E_out for C={c}: {E_out}")

E_out for C=0.01: 0.02358490566037736
E_out for C=1: 0.02122641509433962
E_out for C=100: 0.018867924528301886
E_out for C=10000.0: 0.02358490566037736
E_out for C=1000000.0: 0.02358490566037736
